In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Load your dataset
df = pd.read_csv('/Users/danghoanganhquan/Desktop/Machine Learning/project/data (not featured)/atp_matches_final_cleaned(1991-2024).csv')

# Ensure dates are datetime objects
df['tourney_date'] = pd.to_datetime(df['tourney_date'])

# Sort by date (CRUCIAL to prevent leakage)
df = df.sort_values(['tourney_date', 'match_num']).copy()

In [6]:
required_serve_columns = ['w_1stWon', 'l_1stWon', 'w_2ndWon', 'l_2ndWon']
for col in required_serve_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

#Transform serve stats to player1/player2 format
serve_stats = ['1stWon', '2ndWon']
for stat in serve_stats:
    df[f'player1_{stat}'] = np.where(df['is_player1_winner'], df[f'w_{stat}'], df[f'l_{stat}'])
    df[f'player2_{stat}'] = np.where(df['is_player1_winner'], df[f'l_{stat}'], df[f'w_{stat}'])

# Verify transformation
print("\nVerification sample:")
sample = df[['is_player1_winner', 'w_1stWon', 'l_1stWon', 'player1_1stWon', 'player2_1stWon']].head()
print(sample)

# 4. Calculate serve percentages (if needed)
if all(col in df.columns for col in ['player1_1stIn', 'player1_1stWon']):
    df['player1_1stServeWonPct'] = np.where(df['player1_1stIn'] == 0, 
                                            0, 
                                            df['player1_1stWon'] / df['player1_1stIn'])
    df['player2_1stServeWonPct'] = np.where(df['player2_1stIn'] == 0, 
                                            0, 
                                            df['player2_1stWon'] / df['player2_1stIn'])


Verification sample:
   is_player1_winner  w_1stWon  l_1stWon  player1_1stWon  player2_1stWon
0               True      30.0      17.0            30.0            17.0
1              False      33.0      31.0            31.0            33.0
2              False      17.0      22.0            22.0            17.0
3              False      28.0      23.0            23.0            28.0
4               True      35.0      24.0            35.0            24.0


In [16]:
# Calculate 2nd serve points faced (total service points - 1st serves in)
df['player1_2ndSv'] = df['player1_svpt'] - df['player1_1stIn']
df['player2_2ndSv'] = df['player2_svpt'] - df['player2_1stIn']

# Calculate 2nd serve win percentages (safe division)
df['player1_2ndServeWonPct'] = np.where(
    df['player1_2ndSv'] > 0,
    df['player1_2ndWon'] / df['player1_2ndSv'],
    0  # Default when no 2nd serves faced
)
df['player2_2ndServeWonPct'] = np.where(
    df['player2_2ndSv'] > 0,
    df['player2_2ndWon'] / df['player2_2ndSv'],
    0
)

# Verification
print("\n2nd Serve Performance Verification:")
verification_df = df[[
    'player1_id', 'player2_id',
    'player1_svpt', 'player1_1stIn', 'player1_2ndWon', 'player1_2ndSv', 'player1_2ndServeWonPct',
    'player2_svpt', 'player2_1stIn', 'player2_2ndWon', 'player2_2ndSv', 'player2_2ndServeWonPct'
]].head()

# Add calculated columns to verify math
verification_df = verification_df.assign(
    player1_calc_2ndSv = verification_df['player1_svpt'] - verification_df['player1_1stIn'],
    player2_calc_2ndSv = verification_df['player2_svpt'] - verification_df['player2_1stIn'],
    player1_calc_pct = verification_df['player1_2ndWon'] / verification_df['player1_2ndSv'].replace(0, 1),
    player2_calc_pct = verification_df['player2_2ndWon'] / verification_df['player2_2ndSv'].replace(0, 1)
)

print(verification_df)


2nd Serve Performance Verification:
   player1_id  player2_id  player1_svpt  player1_1stIn  player1_2ndWon  \
0      101142      101746          53.0           37.0             7.0   
1      100656      100923          82.0           47.0            21.0   
2      100587      101613          56.0           30.0             6.0   
3      101511      101196          64.0           48.0             7.0   
4      101179      101601          80.0           55.0            16.0   

   player1_2ndSv  player1_2ndServeWonPct  player2_svpt  player2_1stIn  \
0           16.0                0.437500          46.0           30.0   
1           35.0                0.600000          68.0           47.0   
2           26.0                0.230769          56.0           25.0   
3           16.0                0.437500          48.0           35.0   
4           25.0                0.640000          68.0           43.0   

   player2_2ndWon  player2_2ndSv  player2_2ndServeWonPct  player1_calc_2ndSv  \

In [13]:
required_serve_columns = ['w_1stWon', 'l_1stWon', 'w_2ndWon', 'l_2ndWon']
for col in required_serve_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

#Transform serve stats to player1/player2 format
serve_stats = ['1stWon', '2ndWon']
for stat in serve_stats:
    df[f'player1_{stat}'] = np.where(df['is_player1_winner'], df[f'w_{stat}'], df[f'l_{stat}'])
    df[f'player2_{stat}'] = np.where(df['is_player1_winner'], df[f'l_{stat}'], df[f'w_{stat}'])

# Verify transformation
print("\nVerification sample:")
sample = df[['is_player1_winner', 'w_1stWon', 'l_1stWon', 'player1_1stWon', 'player2_1stWon']].head()
print(sample)

# 4. Calculate serve percentages (if needed)
if all(col in df.columns for col in ['player1_1stIn', 'player1_1stWon']):
    df['player1_1stServeWonPct'] = np.where(df['player1_1stIn'] == 0, 
                                            0, 
                                            df['player1_1stWon'] / df['player1_1stIn'])
    df['player2_1stServeWonPct'] = np.where(df['player2_1stIn'] == 0, 
                                            0, 
                                            df['player2_1stWon'] / df['player2_1stIn'])


Verification sample:
   is_player1_winner  w_1stWon  l_1stWon  player1_1stWon  player2_1stWon
0               True      30.0      17.0            30.0            17.0
1              False      33.0      31.0            31.0            33.0
2              False      17.0      22.0            22.0            17.0
3              False      28.0      23.0            23.0            28.0
4               True      35.0      24.0            35.0            24.0


In [19]:
# Calculate current serve percentages
df['player1_1stServePct'] = df['player1_1stIn'] / df['player1_svpt'].replace(0, np.nan)
df['player2_1stServePct'] = df['player2_1stIn'] / df['player2_svpt'].replace(0, np.nan)

# Calculate global average of first serves
global_avg = df['player1_1stServePct'].mean()

# Initialize dictionaries to track player histories
player_history = defaultdict(list)

# Calculate features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    
    # Player 1 features
    if not player_history[p1]:
        df.at[idx, 'player1_1stServePct_last1'] = global_avg
        df.at[idx, 'player1_1stServePct_last3'] = global_avg
    else:
        df.at[idx, 'player1_1stServePct_last1'] = player_history[p1][-1]
        df.at[idx, 'player1_1stServePct_last3'] = np.mean(player_history[p1][-3:])
    
    # Player 2 features
    if not player_history[p2]:
        df.at[idx, 'player2_1stServePct_last1'] = global_avg
        df.at[idx, 'player2_1stServePct_last3'] = global_avg
    else:
        df.at[idx, 'player2_1stServePct_last1'] = player_history[p2][-1]
        df.at[idx, 'player2_1stServePct_last3'] = np.mean(player_history[p2][-3:])
    
    # Update history AFTER setting features
    player_history[p1].append(row['player1_1stServePct'])
    player_history[p2].append(row['player2_1stServePct'])

# Verification for specific player
sample_player = df['player1_id'].iloc[0]  # Change to any player ID you want to check
player_matches = df[(df['player1_id'] == sample_player) | 
                   (df['player2_id'] == sample_player)]
player_matches = player_matches.sort_values('tourney_date').head(10)

# Build expected output
history = []
output = []
for idx, row in player_matches.iterrows():
    if row['player1_id'] == sample_player:
        current = row['player1_1stServePct']
        last1 = row['player1_1stServePct_last1']
        last3 = row['player1_1stServePct_last3']
    else:
        current = row['player2_1stServePct']
        last1 = row['player2_1stServePct_last1']
        last3 = row['player2_1stServePct_last3']
    
    output.append({
        'match_num': len(output) + 1,
        'current': current,
        'last1': last1,
        'last3': last3
    })
    history.append(current)

# Print verification
print(f"Verification for Player {sample_player}:")
print(pd.DataFrame(output))

Verification for Player 101142:
   match_num   current     last1     last3
0          1  0.698113  0.601478  0.601478
1          2  0.676923  0.698113  0.698113
2          3  0.719298  0.676923  0.687518
3          4  0.758065  0.719298  0.698112
4          5  0.590909  0.758065  0.718095
5          6  0.467742  0.590909  0.689424
6          7  0.638298  0.467742  0.605572
7          8  0.717391  0.638298  0.565650
8          9  0.574074  0.733333  0.751432
9         10  0.803571  0.717391  0.607810


In [22]:
# Initialize trackers
player_1st_serve_history = defaultdict(list)
player_2nd_serve_history = defaultdict(list)

# Calculate global averages
global_1st_serve_won_avg = np.nanmean(np.concatenate([
    df['player1_1stServeWonPct'].dropna().values,
    df['player2_1stServeWonPct'].dropna().values
]))
global_2nd_serve_won_avg = np.nanmean(np.concatenate([
    df['player1_2ndServeWonPct'].dropna().values,
    df['player2_2ndServeWonPct'].dropna().values
]))


# Calculate rolling features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    
    # 1st Serve Features
    for player, col_prefix in [(p1, 'player1'), (p2, 'player2')]:
        history = player_1st_serve_history[player]
        
        # Last 1 game
        if not history:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last1'] = global_1st_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last1'] = history[-1]
        
        # Last 3 games
        if len(history) < 3:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last3'] = np.mean(history) if history else global_1st_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last3'] = np.mean(history[-3:])
        
        # Last 5 games
        if len(history) < 5:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last5'] = np.mean(history) if history else global_1st_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_1stServeWonPct_last5'] = np.mean(history[-5:])
    
    # 2nd Serve Features
    for player, col_prefix in [(p1, 'player1'), (p2, 'player2')]:
        history = player_2nd_serve_history[player]
        
        # Last 1 game
        if not history:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last1'] = global_2nd_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last1'] = history[-1]
        
        # Last 3 games
        if len(history) < 3:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last3'] = np.mean(history) if history else global_2nd_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last3'] = np.mean(history[-3:])
        
        # Last 5 games
        if len(history) < 5:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last5'] = np.mean(history) if history else global_2nd_serve_won_avg
        else:
            df.at[idx, f'{col_prefix}_2ndServeWonPct_last5'] = np.mean(history[-5:])
    
    # Update histories AFTER calculating features
    player_1st_serve_history[p1].append(row['player1_1stServeWonPct'])
    player_1st_serve_history[p2].append(row['player2_1stServeWonPct'])
    player_2nd_serve_history[p1].append(row['player1_2ndServeWonPct'])
    player_2nd_serve_history[p2].append(row['player2_2ndServeWonPct'])

# Calculate differences
for window in [1, 3, 5]:
    df[f'1stServeWonPct_diff_last{window}'] = (
        df[f'player1_1stServeWonPct_last{window}'] - 
        df[f'player2_1stServeWonPct_last{window}']
    )
    df[f'2ndServeWonPct_diff_last{window}'] = (
        df[f'player1_2ndServeWonPct_last{window}'] - 
        df[f'player2_2ndServeWonPct_last{window}']
    )

In [24]:
# Choose player (Federer's ID), for verification purposes
player_id = 104925  

# Filter matches where this player participated
player_matches = df[(df['player1_id'] == player_id) | (df['player2_id'] == player_id)].copy()

# Add position (player1 or player2)
player_matches['position'] = player_matches.apply(
    lambda x: 'player1' if x['player1_id'] == player_id else 'player2', axis=1)

# Get player's stats + historical features
player_matches['1stServeWonPct'] = player_matches.apply(
    lambda x: x['player1_1stServeWonPct'] if x['position'] == 'player1' else x['player2_1stServeWonPct'], axis=1)
player_matches['2ndServeWonPct'] = player_matches.apply(
    lambda x: x['player1_2ndServeWonPct'] if x['position'] == 'player1' else x['player2_2ndServeWonPct'], axis=1)

# Historical features (last 1/3/5 matches)
for serve_type in ['1st', '2nd']:
    for window in [1, 3, 5]:
        player_matches[f'{serve_type}ServeWonPct_last{window}'] = player_matches.apply(
            lambda x: x[f'player1_{serve_type}ServeWonPct_last{window}'] if x['position'] == 'player1' 
                      else x[f'player2_{serve_type}ServeWonPct_last{window}'], axis=1)

# Print simplified view
print(f"\nFirst 10 matches for player {player_id} (Federer):")
cols_to_show = [

    '1stServeWonPct', '1stServeWonPct_last1', '1stServeWonPct_last3', '1stServeWonPct_last5'
]

print(player_matches[cols_to_show].head(10).to_string(float_format='%.3f'))


First 10 matches for player 104925 (Federer):
       1stServeWonPct  1stServeWonPct_last1  1stServeWonPct_last3  1stServeWonPct_last5
41555           0.500                 0.711                 0.711                 0.711
42050           0.650                 0.500                 0.500                 0.500
42077           0.597                 0.650                 0.575                 0.575
42120           0.643                 0.597                 0.582                 0.582
42729           0.410                 0.643                 0.630                 0.597
43423           0.707                 0.410                 0.550                 0.560
43877           0.744                 0.707                 0.587                 0.601
43923           0.548                 0.744                 0.620                 0.620
44169           0.815                 0.548                 0.666                 0.610
44206           0.714                 0.815                 0.702        

In [27]:
# Calculate current match break point stats
# 1. Calculate Break Points Saved Percentage with intelligent imputation
df['player1_bpSavedPct'] = np.where(
    df['player1_bpFaced'] > 0,
    df['player1_bpSaved'] / df['player1_bpFaced'],
    1.0  # Impute 1.0 (100%) when faced 0 break points
)
df['player2_bpSavedPct'] = np.where(
    df['player2_bpFaced'] > 0,
    df['player2_bpSaved'] / df['player2_bpFaced'],
    1.0  # Impute 1.0 (100%) when faced 0 break points
)

# 2. Break Points Faced Per Service Point (alternative since we don't have games)
df['player1_bpPerSvcPt'] = df['player1_bpFaced'] / df['player1_svpt']
df['player2_bpPerSvcPt'] = df['player2_bpFaced'] / df['player2_svpt']

# Initialize trackers
player_bp_saved_history = defaultdict(list)
player_bp_per_svcpt_history = defaultdict(list)

# Calculate global averages
global_bp_saved_avg = np.nanmean(np.concatenate([
    df['player1_bpSavedPct'].dropna().values,
    df['player2_bpSavedPct'].dropna().values
]))
global_bp_per_svcpt_avg = np.nanmean(np.concatenate([
    df['player1_bpPerSvcPt'].dropna().values,
    df['player2_bpPerSvcPt'].dropna().values
]))


# Calculate rolling features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    
    # Break Points Saved %
    for player, col_prefix in [(p1, 'player1'), (p2, 'player2')]:
        saved_history = player_bp_saved_history[player]
        per_svcpt_history = player_bp_per_svcpt_history[player]
        
        # BP Saved % features
        for window in [1, 3, 5]:
            col_name = f'{col_prefix}_bpSavedPct_last{window}'
            if len(saved_history) < window:
                df.at[idx, col_name] = np.mean(saved_history) if saved_history else global_bp_saved_avg
            else:
                df.at[idx, col_name] = np.mean(saved_history[-window:])
        
        # BP Per Service Point features
        for window in [1, 3, 5]:
            col_name = f'{col_prefix}_bpPerSvcPt_last{window}'
            if len(per_svcpt_history) < window:
                df.at[idx, col_name] = np.mean(per_svcpt_history) if per_svcpt_history else global_bp_per_svcpt_avg
            else:
                df.at[idx, col_name] = np.mean(per_svcpt_history[-window:])
    
    # Update histories AFTER calculating features
    player_bp_saved_history[p1].append(row['player1_bpSavedPct'])
    player_bp_saved_history[p2].append(row['player2_bpSavedPct'])
    player_bp_per_svcpt_history[p1].append(row['player1_bpPerSvcPt'])
    player_bp_per_svcpt_history[p2].append(row['player2_bpPerSvcPt'])

# Calculate differences
for window in [1, 3, 5]:
    df[f'bpSavedPct_diff_last{window}'] = df[f'player1_bpSavedPct_last{window}'] - df[f'player2_bpSavedPct_last{window}']
    df[f'bpPerSvcPt_diff_last{window}'] = df[f'player1_bpPerSvcPt_last{window}'] - df[f'player2_bpPerSvcPt_last{window}']

In [29]:
#for verification purposes
player_id = 104925  # Federer

# Filter and prepare player data
player_matches = df[(df['player1_id'] == player_id) | (df['player2_id'] == player_id)].copy()
player_matches['position'] = player_matches.apply(
    lambda x: 'player1' if x['player1_id'] == player_id else 'player2', axis=1)

# Get player's actual stats and features
stats = {
    'bpSavedPct': lambda x: x['player1_bpSavedPct'] if x['position'] == 'player1' else x['player2_bpSavedPct'],
    'bpPerSvcPt': lambda x: x['player1_bpPerSvcPt'] if x['position'] == 'player1' else x['player2_bpPerSvcPt']
}

for stat in stats:
    player_matches[stat] = player_matches.apply(stats[stat], axis=1)
    for window in [1, 3, 5]:
        player_matches[f'{stat}_last{window}'] = player_matches.apply(
            lambda x: x[f'player1_{stat}_last{window}'] if x['position'] == 'player1' 
                      else x[f'player2_{stat}_last{window}'], axis=1)

# Display
cols = [
        'bpSavedPct', 'bpSavedPct_last1', 'bpSavedPct_last3', 'bpSavedPct_last5',
        'bpPerSvcPt', 'bpPerSvcPt_last1', 'bpPerSvcPt_last3', 'bpPerSvcPt_last5']

print(f"Break Point Features for Player {player_id} (Federer):")
print(player_matches[cols].head(10).to_string(float_format='%.3f'))

Break Point Features for Player 104925 (Federer):
       bpSavedPct  bpSavedPct_last1  bpSavedPct_last3  bpSavedPct_last5  bpPerSvcPt  bpPerSvcPt_last1  bpPerSvcPt_last3  bpPerSvcPt_last5
41555       0.250             0.598             0.598             0.598       0.123             0.087             0.087             0.087
42050       0.400             0.250             0.250             0.250       0.055             0.123             0.123             0.123
42077       0.455             0.400             0.325             0.325       0.112             0.055             0.089             0.089
42120       0.667             0.455             0.368             0.368       0.127             0.112             0.097             0.097
42729       0.308             0.667             0.507             0.443       0.197             0.127             0.098             0.104
43423       0.500             0.308             0.476             0.416       0.145             0.197             0.145   

In [32]:
# Calculate current match percentages
df['player1_acePct'] = df['player1_ace'] / df['player1_svpt']
df['player2_acePct'] = df['player2_ace'] / df['player2_svpt']

df['player1_dfPct'] = df['player1_df'] / df['player1_svpt']
df['player2_dfPct'] = df['player2_df'] / df['player2_svpt']

# Initialize trackers
player_ace_history = defaultdict(list)
player_df_history = defaultdict(list)

# Calculate global averages
global_ace_avg = np.nanmean(np.concatenate([
    df['player1_acePct'].dropna().values,
    df['player2_acePct'].dropna().values
]))
global_df_avg = np.nanmean(np.concatenate([
    df['player1_dfPct'].dropna().values,
    df['player2_dfPct'].dropna().values
]))


# Calculate rolling features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    
    # Ace Percentage Features
    for player, col_prefix in [(p1, 'player1'), (p2, 'player2')]:
        ace_history = player_ace_history[player]
        df_history = player_df_history[player]
        
        # Ace % features
        for window in [1, 3, 5]:
            if len(ace_history) < window:
                df.at[idx, f'{col_prefix}_acePct_last{window}'] = np.mean(ace_history) if ace_history else global_ace_avg
            else:
                df.at[idx, f'{col_prefix}_acePct_last{window}'] = np.mean(ace_history[-window:])
        
        # DF % features
        for window in [1, 3, 5]:
            if len(df_history) < window:
                df.at[idx, f'{col_prefix}_dfPct_last{window}'] = np.mean(df_history) if df_history else global_df_avg
            else:
                df.at[idx, f'{col_prefix}_dfPct_last{window}'] = np.mean(df_history[-window:])
    
    # Update histories AFTER calculating features
    player_ace_history[p1].append(row['player1_acePct'])
    player_ace_history[p2].append(row['player2_acePct'])
    player_df_history[p1].append(row['player1_dfPct'])
    player_df_history[p2].append(row['player2_dfPct'])

# Calculate differences
for window in [1, 3, 5]:
    df[f'acePct_diff_last{window}'] = df[f'player1_acePct_last{window}'] - df[f'player2_acePct_last{window}']
    df[f'dfPct_diff_last{window}'] = df[f'player1_dfPct_last{window}'] - df[f'player2_dfPct_last{window}']

In [35]:
player_id = 104925  # Federer

# Filter and prepare player data
player_matches = df[(df['player1_id'] == player_id) | (df['player2_id'] == player_id)].copy()
player_matches['position'] = player_matches.apply(
    lambda x: 'player1' if x['player1_id'] == player_id else 'player2', axis=1)

# Get player's actual stats and features
stats = {
    'acePct': lambda x: x['player1_acePct'] if x['position'] == 'player1' else x['player2_acePct'],
    'dfPct': lambda x: x['player1_dfPct'] if x['position'] == 'player1' else x['player2_dfPct']
}

for stat in stats:
    player_matches[stat] = player_matches.apply(stats[stat], axis=1)
    for window in [1, 3, 5]:
        player_matches[f'{stat}_last{window}'] = player_matches.apply(
            lambda x: x[f'player1_{stat}_last{window}'] if x['position'] == 'player1' 
                      else x[f'player2_{stat}_last{window}'], axis=1)

# Display
cols = [
        'acePct', 'acePct_last1', 'acePct_last3', 'acePct_last5',
        'dfPct', 'dfPct_last1', 'dfPct_last3', 'dfPct_last5']

print(f"Ace/DF Features for Player {player_id} (Federer):")
print(player_matches[cols].head(10).to_string(float_format='%.3f'))

Ace/DF Features for Player 104925 (Federer):
       acePct  acePct_last1  acePct_last3  acePct_last5  dfPct  dfPct_last1  dfPct_last3  dfPct_last5
41555   0.062         0.073         0.073         0.073  0.092        0.039        0.039        0.039
42050   0.044         0.062         0.062         0.062  0.022        0.092        0.092        0.092
42077   0.031         0.044         0.053         0.053  0.010        0.022        0.057        0.057
42120   0.127         0.031         0.045         0.045  0.102        0.010        0.041        0.041
42729   0.015         0.127         0.067         0.066  0.076        0.102        0.045        0.057
43423   0.029         0.015         0.058         0.056  0.058        0.076        0.063        0.060
43877   0.103         0.029         0.057         0.049  0.034        0.058        0.078        0.054
43923   0.037         0.103         0.049         0.061  0.037        0.034        0.056        0.056
44169   0.021         0.037         0

In [38]:
# Calculate simple differences for ranking features
df['rank_diff'] = df['player1_rank'] - df['player2_rank']
df['rank_points_diff'] = df['player1_rank_points'] - df['player2_rank_points']

In [41]:
from collections import defaultdict

# Initialize H2H trackers
h2h_overall = defaultdict(lambda: defaultdict(int))  # h2h_overall[p1][p2] = wins
h2h_surface = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))  # h2h_surface[p1][p2][surface] = wins


# Initialize new columns
df['h2h_diff'] = 0
df['h2h_surface_diff'] = 0

# Calculate H2H features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    surface = row['surface']
    
    # Calculate differences (BEFORE updating trackers to exclude current match)
    df.at[idx, 'h2h_diff'] = h2h_overall[p1][p2] - h2h_overall[p2][p1]
    df.at[idx, 'h2h_surface_diff'] = h2h_surface[p1][p2][surface] - h2h_surface[p2][p1][surface]
    
    # Update trackers AFTER calculating features (current match not included)
    winner = p1 if row['winner_id'] == p1 else p2
    h2h_overall[winner][p2 if winner == p1 else p1] += 1
    h2h_surface[winner][p2 if winner == p1 else p1][surface] += 1

# Verification for a specific player (Federer vs Nadal example)
player1_id = 104925  # Federer
player2_id = 104745  # Nadal

# Filter their matches
rivalry_matches = df[((df['player1_id'] == player1_id) & (df['player2_id'] == player2_id)) | 
                     ((df['player1_id'] == player2_id) & (df['player2_id'] == player1_id))].copy()

# Add perspective columns
rivalry_matches['federer_perspective'] = rivalry_matches.apply(
    lambda x: 1 if x['player1_id'] == player1_id else -1, axis=1)
rivalry_matches['h2h_diff_fed'] = rivalry_matches['h2h_diff'] * rivalry_matches['federer_perspective']
rivalry_matches['h2h_surface_diff_fed'] = rivalry_matches['h2h_surface_diff'] * rivalry_matches['federer_perspective']

# Display relevant columns
cols = ['tourney_date', 'tourney_name', 'surface', 'round', 'winner_id',
        'h2h_diff_fed', 'h2h_surface_diff_fed']

print(f"Head-to-Head Features for Federer (ID:{player1_id}) vs Nadal (ID:{player2_id}):")
print(rivalry_matches[cols].head(10).to_string())

Head-to-Head Features for Federer (ID:104925) vs Nadal (ID:104745):
      tourney_date          tourney_name surface round  winner_id  h2h_diff_fed  h2h_surface_diff_fed
46863   2006-05-29         Roland Garros    Clay    QF     104745             0                     0
49108   2007-03-05  Indian Wells Masters    Hard     F     104745            -1                     0
49199   2007-03-19         Miami Masters    Hard    QF     104925            -2                    -1
49517   2007-05-07          Rome Masters    Clay    QF     104745            -1                    -1
49747   2007-05-28         Roland Garros    Clay    SF     104745            -2                    -2
50014   2007-06-25             Wimbledon   Grass    SF     104745            -3                     0
50439   2007-08-05        Canada Masters    Hard    SF     104925            -4                     0
51149   2007-11-12           Masters Cup    Hard    RR     104745            -3                     1
51900   2008-0

In [44]:
from collections import defaultdict
import pandas as pd

# Initialize trackers - now storing full win/loss history
player_win_history = defaultdict(list)


# Create new columns for individual win counts
for window in [5, 10, 20]:
    df[f'player1_win_last{window}'] = 0
    df[f'player2_win_last{window}'] = 0
    df[f'win_diff_last{window}'] = 0  # Difference columns

# Calculate features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    
    # Get win histories (BEFORE updating)
    p1_history = player_win_history[p1]
    p2_history = player_win_history[p2]
    
    # Calculate individual win counts
    for window in [5, 10, 20]:
        # Player 1 wins (use all available history if < window)
        p1_wins = sum(p1_history[-window:]) if p1_history else 0
        df.at[idx, f'player1_win_last{window}'] = p1_wins
        
        # Player 2 wins (use all available history if < window)
        p2_wins = sum(p2_history[-window:]) if p2_history else 0
        df.at[idx, f'player2_win_last{window}'] = p2_wins
        
        # Calculate difference
        df.at[idx, f'win_diff_last{window}'] = p1_wins - p2_wins
    
    # Update histories AFTER processing current match
    winner = row['winner_id']
    player_win_history[p1].append(1 if winner == p1 else 0)
    player_win_history[p2].append(1 if winner == p2 else 0)

In [46]:
# Print first 20 matches with all win streak features
print("First 20 Matches with Win Streak Features:")
print(df[[
    'player1_name', 'player1_win_last5', 'player1_win_last10', 'player1_win_last20',
]].head(20).to_string())

First 20 Matches with Win Streak Features:
              player1_name  player1_win_last5  player1_win_last10  player1_win_last20
0           Emilio Sanchez                  0                   0                   0
1               Ivan Lendl                  0                   0                   0
2                Steve Guy                  0                   0                   0
3              Horst Skoff                  0                   0                   0
4   Jean Philippe Fleurian                  0                   0                   0
5             Martin Jaite                  0                   0                   0
6               Eric Jelen                  0                   0                   0
7            David Wheaton                  0                   0                   0
8         Richard Fromberg                  0                   0                   0
9              Scott Davis                  0                   0                   0
10     Chri

In [50]:
# Initialize trackers
player_top_wins = {
    'top10': defaultdict(list),  # Stores wins against top 10 opponents
    'top20': defaultdict(list)   # Stores wins against top 20 opponents
}

# Create new columns
for rank_range in ['top10', 'top20']:
    for window in [5, 10]:
        df[f'{rank_range}_win_diff_last{window}'] = 0

# Calculate features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    p1_rank, p2_rank = row['player1_rank'], row['player2_rank']
    
    # Determine if opponents are top-ranked (BEFORE updating trackers)
    p2_is_top10 = 1 if p2_rank <= 10 else 0
    p2_is_top20 = 1 if p2_rank <= 20 else 0
    p1_is_top10 = 1 if p1_rank <= 10 else 0
    p1_is_top20 = 1 if p1_rank <= 20 else 0
    
    # Calculate differences for each window
    for window in [5, 10]:
        # Top 10 wins difference
        p1_top10_wins = sum(player_top_wins['top10'][p1][-window:])
        p2_top10_wins = sum(player_top_wins['top10'][p2][-window:])
        df.at[idx, f'top10_win_diff_last{window}'] = p1_top10_wins - p2_top10_wins
        
        # Top 20 wins difference
        p1_top20_wins = sum(player_top_wins['top20'][p1][-window:])
        p2_top20_wins = sum(player_top_wins['top20'][p2][-window:])
        df.at[idx, f'top20_win_diff_last{window}'] = p1_top20_wins - p2_top20_wins
    
    # Update trackers AFTER calculating features (current match not included in next calculations)
    winner = row['winner_id']
    
    # Update top10 tracker
    if p2_rank <= 10:  # If p2 is top10
        player_top_wins['top10'][p1].append(1 if winner == p1 else 0)
    if p1_rank <= 10:  # If p1 is top10
        player_top_wins['top10'][p2].append(1 if winner == p2 else 0)
    
    # Update top20 tracker
    if p2_rank <= 20:  # If p2 is top20
        player_top_wins['top20'][p1].append(1 if winner == p1 else 0)
    if p1_rank <= 20:  # If p1 is top20
        player_top_wins['top20'][p2].append(1 if winner == p2 else 0)

/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/3571738560.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{rank_range}_win_diff_last{window}'] = 0


In [52]:
# Verification for a player (Federer)
player_id = 104925
verif_matches = df[((df['player1_id'] == player_id) | (df['player2_id'] == player_id))].head(20).copy()

# Add perspective columns
verif_matches['position'] = verif_matches.apply(
    lambda x: 'player1' if x['player1_id'] == player_id else 'player2', axis=1)

# Calculate player's top win counts
for window in [5, 10]:
    verif_matches[f'player_top10_wins_last{window}'] = verif_matches.apply(
        lambda x: sum(player_top_wins['top10'][player_id][-window:]), axis=1)
    verif_matches[f'player_top20_wins_last{window}'] = verif_matches.apply(
        lambda x: sum(player_top_wins['top20'][player_id][-window:]), axis=1)

# Display verification
cols = ['player1_rank', 'player2_rank']
for window in [5, 10]:
    cols.extend([
        f'player_top10_wins_last{window}',
        f'top10_win_diff_last{window}'
    ])

print("Top Rank Win Difference Verification:")
print(verif_matches[cols].to_string())

Top Rank Win Difference Verification:
       player1_rank  player2_rank  player_top10_wins_last5  top10_win_diff_last5  player_top10_wins_last10  top10_win_diff_last10
41555          65.0         368.0                        2                     1                         5                      1
42050          67.0         272.0                        2                     0                         5                      1
42077         272.0          54.0                        2                    -2                         5                     -3
42120         248.0          81.0                        2                     0                         5                      0
42729         188.0           4.0                        2                    -3                         5                     -4
43423         114.0         160.0                        2                     1                         5                      1
43877         153.0          71.0                   

In [56]:
import pandas as pd
from collections import defaultdict
import numpy as np

# Initialize surface Elo ratings (starting at 400 for each surface)
surface_elos = defaultdict(lambda: defaultdict(lambda: 400))
K_FACTOR = 32  # Standard Elo update factor

# Initialize Elo change trackers
elo_change_history = defaultdict(lambda: defaultdict(list))

# Create new columns for raw Elo values
df['player1_surface_elo'] = np.nan
df['player2_surface_elo'] = np.nan

# Create Elo difference and change features
for window in [3, 5, 10]:
    df[f'player1_elo_change_last{window}'] = 0
    df[f'player2_elo_change_last{window}'] = 0
    df[f'surface_elo_avg_change_diff_last{window}'] = 0

# Calculate Elo features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    surface = row['surface']
    
    # Get pre-match Elos (BEFORE any updates)
    p1_elo = surface_elos[p1][surface]
    p2_elo = surface_elos[p2][surface]
    
    # Store raw Elo values
    df.at[idx, 'player1_surface_elo'] = p1_elo
    df.at[idx, 'player2_surface_elo'] = p2_elo
    
    # Calculate Elo difference
    df.at[idx, 'surface_elo_diff'] = p1_elo - p2_elo
    
    # Calculate average Elo changes (use available history)
    for window in [3, 5, 10]:
        p1_changes = elo_change_history[p1][surface]
        p2_changes = elo_change_history[p2][surface]
        
        # Player 1 changes (use all available history if < window)
        p1_avg = np.mean(p1_changes[-window:]) if p1_changes else 0
        df.at[idx, f'player1_elo_change_last{window}'] = p1_avg
        
        # Player 2 changes (use all available history if < window)
        p2_avg = np.mean(p2_changes[-window:]) if p2_changes else 0
        df.at[idx, f'player2_elo_change_last{window}'] = p2_avg
        
        # Difference in changes
        df.at[idx, f'surface_elo_avg_change_diff_last{window}'] = p1_avg - p2_avg
    
    # Calculate Elo updates (AFTER feature calculation)
    winner = row['winner_id']
    actual_score = 1 if winner == p1 else 0
    
    # Expected scores
    q1 = 10 ** (p1_elo / 400)
    q2 = 10 ** (p2_elo / 400)
    expected_p1 = q1 / (q1 + q2)
    
    # Calculate Elo updates
    p1_update = K_FACTOR * (actual_score - expected_p1)
    p2_update = K_FACTOR * ((1 - actual_score) - (1 - expected_p1))
    
    # Store updates but don't apply until next match
    elo_change_history[p1][surface].append(p1_update)
    elo_change_history[p2][surface].append(p2_update)
    
    # Update Elos for future matches (not current match!)
    surface_elos[p1][surface] += p1_update
    surface_elos[p2][surface] += p2_update

/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/4255063847.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player1_surface_elo'] = np.nan
/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/4255063847.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player2_surface_elo'] = np.nan
/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/4255063847.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

In [59]:
# Print first 20 matches with  Elo features for verification
print("First 20 Matches with Elo Features:")
print(df[[
    'surface',
    'player1_name', 'player1_surface_elo',
    'player2_name', 'player2_surface_elo',
    'surface_elo_diff'
]].head(20).to_string(float_format='%.1f'))

First 20 Matches with Elo Features:
   surface            player1_name  player1_surface_elo        player2_name  player2_surface_elo  surface_elo_diff
0     Hard          Emilio Sanchez                400.0        Renzo Furlan                400.0               0.0
1     Hard              Ivan Lendl                400.0         Wally Masur                400.0               0.0
2     Hard               Steve Guy                400.0  Malivai Washington                400.0               0.0
3     Hard             Horst Skoff                400.0    Derrick Rostagno                400.0               0.0
4     Hard  Jean Philippe Fleurian                400.0        Brett Steven                400.0               0.0
5     Hard            Martin Jaite                400.0     Guillaume Raoux                400.0               0.0
6     Hard              Eric Jelen                400.0         Gilad Bloom                400.0               0.0
7     Hard           David Wheaton          

In [62]:
# First calculate the average minutes played from non-missing values
avg_minutes = df['minutes'].mean()

# Impute missing minutes with the average
df['minutes'] = df['minutes'].fillna(avg_minutes)

# Then proceed with your existing hours_diff feature creation code
# (This would be where you calculate hrs_diff_prev1, hrs_diff_prev2, etc.)

In [65]:
# Initialize tracker for tournament hours played
tournament_hours = defaultdict(lambda: defaultdict(list))  # tournament_hours[tourney_id][player_id] = [hrs1, hrs2]

# Create new columns
df['player1_hrs_prev1'] = 0.0  # Hours in 1 match back
df['player1_hrs_prev2'] = 0.0  # Hours in 2 matches back
df['player2_hrs_prev1'] = 0.0
df['player2_hrs_prev2'] = 0.0
df['hrs_diff_prev1'] = 0.0  # player1 - player2 (1 match back)
df['hrs_diff_prev2'] = 0.0  # player1 - player2 (2 matches back)

# Convert minutes to hours (if minutes exist, else assume 1.5 hours as default)
if 'minutes' in df.columns:
    df['hours_played'] = df['minutes'] / 60
else:
    df['hours_played'] = 1.5  # Default to 1.5 hours if data missing

# Calculate features
for idx, row in df.iterrows():
    p1, p2 = row['player1_id'], row['player2_id']
    tourney_id = row['tourney_id']
    hours = row['hours_played']
    
    # Get player1's previous hours (convert minutes to hours if needed)
    p1_prev_hrs = tournament_hours[tourney_id][p1]
    p1_hrs_prev1 = p1_prev_hrs[-1] if p1_prev_hrs else 0.0
    p1_hrs_prev2 = p1_prev_hrs[-2] if len(p1_prev_hrs) > 1 else p1_hrs_prev1
    
    # Get player2's previous hours
    p2_prev_hrs = tournament_hours[tourney_id][p2]
    p2_hrs_prev1 = p2_prev_hrs[-1] if p2_prev_hrs else 0.0
    p2_hrs_prev2 = p2_prev_hrs[-2] if len(p2_prev_hrs) > 1 else p2_hrs_prev1
    
    # Store features (BEFORE updating)
    df.at[idx, 'player1_hrs_prev1'] = p1_hrs_prev1
    df.at[idx, 'player1_hrs_prev2'] = p1_hrs_prev2
    df.at[idx, 'player2_hrs_prev1'] = p2_hrs_prev1
    df.at[idx, 'player2_hrs_prev2'] = p2_hrs_prev2
    df.at[idx, 'hrs_diff_prev1'] = p1_hrs_prev1 - p2_hrs_prev1
    df.at[idx, 'hrs_diff_prev2'] = p1_hrs_prev2 - p2_hrs_prev2
    
    # Update tracker AFTER processing current match
    tournament_hours[tourney_id][p1].append(hours)
    tournament_hours[tourney_id][p2].append(hours)

/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/3601274055.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player1_hrs_prev1'] = 0.0  # Hours in 1 match back
/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/3601274055.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player1_hrs_prev2'] = 0.0  # Hours in 2 matches back
/var/folders/0m/y6g4lb3s2ts7wd2gkzhjtjwm0000gn/T/ipykernel_76013/3601274055.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [67]:
# Verification - show first tournament's matches
sample_tourney = df['tourney_id'].iloc[0]
verif_matches = df[df['tourney_id'] == sample_tourney].head(20).copy()

print("Hours Played Features Verification:")
print(verif_matches[[
    'player1_name', 'player1_hrs_prev1', 'player1_hrs_prev2',
    'player2_name',
    'hours_played'
]].to_string(float_format='%.2f'))

Hours Played Features Verification:
              player1_name  player1_hrs_prev1  player1_hrs_prev2            player2_name  hours_played
0           Emilio Sanchez               0.00               0.00            Renzo Furlan          1.05
2                Steve Guy               0.00               0.00      Malivai Washington          1.20
4   Jean Philippe Fleurian               0.00               0.00            Brett Steven          1.68
6               Eric Jelen               0.00               0.00             Gilad Bloom          1.80
8         Richard Fromberg               0.00               0.00             Chuck Adams          1.08
10     Christian Bergstrom               0.00               0.00          Javier Sanchez          0.87
12          Kelly Evernden               0.00               0.00             Jaime Yzaga          1.43
14             Luiz Mattar               0.00               0.00             David Engel          2.30
16          Omar Camporese           

In [70]:
import pandas as pd

# Display all columns with missing value counts
print("DataFrame Columns with Missing Values:")
print(pd.DataFrame({
    'Column': df.columns,
    'Missing Values': df.isnull().sum(),
    'Percentage Missing': (df.isnull().mean() * 100).round(2)
}).to_string())

DataFrame Columns with Missing Values:
                                                                Column  Missing Values  Percentage Missing
tourney_id                                                  tourney_id               0                0.00
tourney_name                                              tourney_name               0                0.00
surface                                                        surface               0                0.00
draw_size                                                    draw_size               0                0.00
tourney_level                                            tourney_level               0                0.00
tourney_date                                              tourney_date               0                0.00
match_num                                                    match_num               0                0.00
winner_id                                                    winner_id               0                0.0